In [52]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

El votante representativo se preocupa por el valor esperado de su función de utilidad  $\mathbb{E}_{t}^{P}(\Gamma_{t})$

<ul>
    <li>c representa el consumo del bien privado por parte de un ciudadano representativo.</li>
    <li>g es el consumo del bien público (per cápita)</li>
    <li>k es la inversión pública (per cápita)</li>
</ul>

Función de utilidad del líder $\mathbf{E}_{t}^{I}(\Gamma_{t})+\sum_{s=t}^{T}\beta^{s-t}X\pi_{s,t}$

Función de elección del líder

\begin{equation}
    v_{t} =
    \begin{cases}
        1 &  \text{si } \mathbf{E}_{t}^{P}(\Gamma_{t+1}) \geq \mathbf{E}_{t}^{P}(\Gamma_{t+1}^{O})\\
        0 & \text{en otro caso}\\
\end{cases}
\end{equation}

## Equilibrio bajo información completa

\begin{equation}
    \begin{aligned}
        \max_{\tau, g} & U(y-\tau, g)+\beta V(\tau + \epsilon - g) \\
        \text{sujeto a } & g, y- \tau, \tau + \epsilon-g \geq 0 \\
   \end{aligned}
\end{equation}

\begin{equation}
    \begin{aligned}
        \max_{\tau, g} & (y-\tau, g)+\beta V(\tau + \epsilon - g) \\
        \text{sujeto a } & g, y- \tau, \tau + \epsilon-g \geq 0 \\
   \end{aligned}
\end{equation}

### En $t=\bar{t}$

In [53]:
# Parámetros
a, b = 1, 1
beta = 0.95

y = 100  # Valor exógeno

$$ \epsilon_{t}^{i} = \alpha_{t}^{i} + \alpha_{t-1}^{i}$$
$$ \eta_{t}^{i} = q_{t}^{i} + q_{t-1}^{i}$$

$$U(c, g) = a \ln{c} + b\ln{g}$$
$$V(k) = -\ln{k}$$

### Problema de maximizacion en T periodos de tiempo

In [54]:
# Parametros
T = 10
y = 100
t = 0
beta = 0.95

a, b = 1, 1

In [55]:
# Probabilidad de que el agente i sea competente en el periodo t 
p = 0.5
# Rango de valores donde viven los shocks de apariencia
q_r = range(-10,10) 

In [56]:
# Valores aleatorios
alpha1_i = np.random.binomial(1, p, T-t)
alpha2_i = np.random.binomial(1, p, T-t)

alpha1_o = np.random.binomial(1, p, T-t)
alpha2_o = np.random.binomial(1, p, T-t)

epsilon_i = alpha1_i + alpha2_i # competencia del Incumbente independiente entre agentes y en el tiempo
epsilon_o = alpha1_o + alpha2_o # competencia del Oponente independiente entre agentes y en el tiempo

# -------------------------------------------------------------------------------------------------------------------
q1_i = np.random.choice(q_r, T-t)
q2_i = np.random.choice(q_r, T-t)

q1_o = np.random.choice(q_r, T-t)
q2_o = np.random.choice(q_r, T-t)

eta_i = q1_i + q2_i # Shock no relacionado con la competencia pero si con el liderazgo del Incumbente independiente entre agentes y en el tiempo
eta_o = q1_o + q2_o # Shock no relacionado con la competencia pero si con el liderazgo del Oponente independiente entre agentes y en el tiempo

In [57]:
# Definimos la funcion objetivo
def funcion_objetivo(x, y, epsilon, beta, a, b):
    tau, g = x
    c = y - tau
    k = tau + epsilon - g
    return -(a * np.log(c) + b * np.log(g) + beta * (-np.log(k)))  # Negativo porque usamos un minimizador de scipy

# Restricciones (Constraints)
def constraint1(x, y):
    # y - tau >= 0
    return y - x[0]

def constraint2(x):
    # g >= 0
    return x[1]

def constraint3(x, epsilon):
    # tau + epsilon - g >= 0
    return x[0] + epsilon - x[1]

In [58]:
# Guess inicial (Por ahora tau y g empiezan tomando valores en la mitad de su rango permitido)
x0 = [y/2, y/4]

______________________

## Parte dinamica

In [59]:
# Funcion para optimizar en cada periodo de tiempo
def optimizador_cada_t(T, y, beta, a, b, epsilon):
    tau_optimo = []
    g_optimo = []
    valores_max = []

    for t in range(T):
        # Itera sobre los valores de competencia del agente
        epsilon_t = epsilon[t]

        # Restricciones por cada periodo de tiempo(epsilon)
        con1 = {'type': 'ineq', 'fun': constraint1, 'args': (y,)}
        con2 = {'type': 'ineq', 'fun': constraint2}
        con3 = {'type': 'ineq', 'fun': constraint3, 'args': (epsilon_t,)}

        # Combinamos las restricciones en una lista
        constraints_t = [con1, con2, con3]

        # Optimiza para ese periodo de tiempo t
        resultado_t = minimize(funcion_objetivo, x0, args=(y, epsilon_t, beta, a, b), constraints=constraints_t)

        # Guarda los resultados para cada periodo de tiempo
        tau_optimo.append(resultado_t.x[0])
        g_optimo.append(resultado_t.x[1])
        valores_max.append(-resultado_t.fun)

    return tau_optimo, g_optimo, valores_max

In [60]:
# Optimiza T veces para el incumbente
tau_i_optimos_lista, g_i_optimos_lista, utilidades_i_maximos = optimizador_cada_t(T, y, beta, a, b, epsilon_i)

# Valor esperado
tau_i_esperado = np.mean(tau_i_optimos_lista)
g_i_esperado = np.mean(g_i_optimos_lista)

utilida_esperada_i = np.mean(utilidades_i_maximos)

C:\Users\oscar\AppData\Local\Temp\ipykernel_10184\1139613495.py:6: RuntimeWarning: invalid value encountered in log
  return -(a * np.log(c) + b * np.log(g) + beta * (-np.log(k)))  # Negativo porque usamos un minimizador de scipy


### Modelo con asimetría de información|

Si los votantes observan el último shock de competencia del incumbente antes de votar $\alpha_{i}$, entonces las expectativas hacia el incumbente serian.
$$ \mathbf{E}_{t}^{P}[W^{*}(\epsilon_{t+1})|\alpha_{t}=\alpha^{i}]=\Omega^{i}$$
Donde $i=H, L$ para los tipos compententes y no competentes en un modelo de senalizacion 

Para el caso del oponente, no se observa cuan competente es, entonces:
$$\mathbf{E}_{t}^{P}[W^{*}(\epsilon_{t+1}^{O})]=\Omega^{O}$$


El problema de maximizacion del incumbente de tipo $i=H, L$ seria:
$$\begin{equation}
    \begin{aligned}
        \max_{\tau, g} & \beta[X(1+\beta) + \Omega^{i}-\Omega^{O}]\pi[\hat{\rho}(g,\tau)]+ W(g, \tau, \epsilon^{i}) \\
        \text{sujeto a } & g, y- \tau, \tau + \epsilon^{i}-g \geq 0 \\
    \end{aligned}
    \end{equation}$$

Para la probabilidad de ganar elegimos una función que toma valores entre 0 y 1 con una relación positiva en g y una negativa en tau para reflejar las preferencias de los agentes por bajos impuestos y alto consumo público, al hacer esto le permitimos al agente manipular $\tau$ y $g$ pero al no modelar el valor esperado de la funcion de votos estariamos tambien quitandole cierta logica al agente racional que no estaria comparando sus utilidades esperadas con el incumbente y con el nuevo.

$$\begin{equation}
    \begin{aligned}
        \pi[\hat{\rho}(\tau, g)] = \mathbf{E}^{I}(v|g, \tau)
    \end{aligned}
    \end{equation}$$


$$\begin{equation}
    \begin{aligned}
        \pi[\hat{\rho}(\tau, g)] = \frac{1}{1+e^{dg-h\tau+m}}
    \end{aligned}
    \end{equation}$$

Donde: $d$, $h$ y $m$ son valores positivos que controlan la sensibilidad de la funcion logistica elegida, calibrar estos valores para reflejar la probabilidad de ganar del incumbente tambien sera un ejercicio clave que podria hacerse a traves de las encuestas, la funcion $\Gamma$ debe ser utilizada para determinar la funcion de votacion comparando esas utilidades esperadas, solo de esa manera se inserta dentro la probabilidad, igual se podria usar esta probabilidad

In [61]:
funcion_objetivo(x0, 100, epsilon_i[t], beta, a, b)

-4.072966796671556

In [62]:
def funcion_objetivo(x, y, epsilon, beta, a, b):
    tau, g = x
    c = y - tau
    k = tau + epsilon - g
    return -(a * np.log(c) + b * np.log(g) + beta * (-np.log(k)))  # Negativo porque usamos un minimizador de scipy

def gamma(x, y, eta, beta, epsilon, t, T, a, b):
    tau, g = x
    c = y - tau
    k = tau + epsilon - g
    return sum([(a * np.log(c) + b * np.log(g) + (-np.log(k[s])) + eta[s])*beta**(s-t) for s in range(t, T)])

def pi(x, d, h, eta_i):
    tau, g = x
    return 1/(1+np.exp(d*g-h*tau+eta_i[t]))


In [64]:
# Funcion para optimizar en cada periodo de tiempo
def optimizador_cada_t(T, y, beta, a, b, epsilon):
    tau_optimo = []
    g_optimo = []
    valores_max = []

    for t in range(T):
        # Itera sobre los valores de competencia del agente
        epsilon_t = epsilon[t]

        # Restricciones por cada periodo de tiempo(epsilon)
        con1 = {'type': 'ineq', 'fun': constraint1, 'args': (y,)}
        con2 = {'type': 'ineq', 'fun': constraint2}
        con3 = {'type': 'ineq', 'fun': constraint3, 'args': (epsilon_t,)}

        # Combinamos las restricciones en una lista
        constraints_t = [con1, con2, con3]

        # Optimiza para ese periodo de tiempo t
        resultado_t = minimize(funcion_objetivo, x0, args=(y, epsilon_t, beta, a, b), constraints=constraints_t)

        # Guarda los resultados para cada periodo de tiempo
        tau_optimo.append(resultado_t.x[0])
        g_optimo.append(resultado_t.x[1])
        valores_max.append(-resultado_t.fun)

    return tau_optimo, g_optimo, valores_max

El problema de maximizacion del incumbente de tipo $i=H, L$ seria:
$$\begin{equation}
    \begin{aligned}
        \max_{\tau, g} & \beta[X(1+\beta) + \Omega^{i}-\Omega^{O}]\pi[\hat{\rho}(g,\tau)]+ W(g, \tau, \epsilon^{i}) \\
        \text{sujeto a } & g, y- \tau, \tau + \epsilon^{i}-g \geq 0 \\
    \end{aligned}
    \end{equation}$$

In [65]:
def asimetria(x, y, a, b, t, T, beta, epsilon_i, epsilon_o, X, d, h, eta_i):
    lista_tau_i, lista_g_i, lista_utilidades_i = optimizador_cada_t(T, y, beta, a, b, epsilon_i)
    lista_tau_o, lista_g_o, lista_utilidades_o = optimizador_cada_t(T, y, beta, a, b, epsilon_o)
    valor_pi = pi(x, d, h, eta_i[t])  # puede ser cambiado por alguna expresión que compare los resultados de la función Gamma
    w_i = -funcion_objetivo(x, y, epsilon_i[t], beta, a, b)

    return (beta*(X*(1+beta) + np.mean(lista_utilidades_i)-np.mean(lista_utilidades_o))*valor_pi + w_i)

In [66]:
asimetria(x0, y, a, b, T, beta=beta, epsilon_i=epsilon_i, epsilon_o=epsilon_o, X=1, d=0.5, h=0.2, eta_i=eta_i)

TypeError: asimetria() missing 1 required positional argument: 'T'

In [67]:
def optimizacion(x, *args):
    return -asimetria(x, *args)

# Argumentos adicionales para asimetria, excepto x
args = (y, a, b, T, beta, epsilon_i, epsilon_o, 1, 0.5, 0.2, 0)

# Valor inicial para x (tau, g)
x0 = [y/2, y/4]

# Restricciones
def restriccion1(x, y):
    return y - x[0]  # y - tau >= 0

def restriccion2(x):
    return x[1]  # g >= 0

def restriccion3(x, epsilon_i):
    return x[0] + epsilon_i - x[1]  # tau + epsilon_i - g >= 0

# Definición de restricciones
constraints = (
    {'type': 'ineq', 'fun': restriccion1, 'args': (y,)},
    {'type': 'ineq', 'fun': restriccion2},
    {'type': 'ineq', 'fun': restriccion3, 'args': (epsilon_i,)}  # Aquí epsilon_i es un escalar
)


# Realizar la optimización
resultado = minimize(optimizacion, x0, args=args, constraints=constraints)

# Resultados óptimos
tau_optimo, g_optimo = resultado.x

TypeError: asimetria() missing 1 required positional argument: 'eta_i'

In [68]:
tau_optimo

36.84442085119685

In [40]:
y

100

In [41]:
tau_optimo + epsilon_i[t] - g_optimo

5.336175945558352e-12

In [42]:

g_optimo

36.844420851191515